<a href="https://colab.research.google.com/github/p16111121/fcc_predict_health_costs_with_regression/blob/main/Copy_of_fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler

dataset = pd.get_dummies(dataset, columns=['sex', 'smoker', 'region'])

dataset_label = dataset['expenses']
dataset_data = dataset.drop('expenses', axis=1)
train_dataset, test_dataset, train_label, test_label = model_selection.train_test_split(dataset_data, dataset_label, test_size = 0.2, random_state = 1234)

scaler = StandardScaler()
train_dataset = scaler.fit_transform(train_dataset)
test_dataset = scaler.transform(test_dataset)

In [ ]:
model = keras.Sequential([
    layers.Dense(32, activation='relu',input_dim = train_dataset.shape[1]),
    layers.Dense(64, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(1)
  ])
model.compile(
    optimizer='adam',
    loss='mean_squared_error',  # 損失函數設為 MSE
    metrics=['mean_absolute_error', 'mean_squared_error']  # 指標包括 MAE 和 MSE
)
model.summary()

In [ ]:
history = model.fit(train_dataset, train_label, epochs=100, batch_size=32, validation_split=0.2)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_label, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_label, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
